In [30]:
from unicodedata import bidirectional
import torch

class TransformerModel(torch.nn.Module):
    def __init__(self, input_size=100, embedding_dim=200) -> None:
        super(TransformerModel, self).__init__()
        self.input_size = input_size
        encoder_layer = torch.nn.TransformerEncoderLayer(d_model=input_size, nhead=5)
        self.tranformer = torch.nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(input_size, 100),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.33),
            torch.nn.Linear(100, 50),
            torch.nn.ReLU(),
            torch.nn.Linear(50,3)
        )

    def forward(self, x):
        x = self.tranformer(x)
        x = x.mean(dim=0)
        x = self.fc(x)
        return x

model = TransformerModel(input_size=100)

In [10]:
from dataset import SentimentAnalysisDataset

train_dataset = SentimentAnalysisDataset('data/trainEmotions.csv')
x,y = train_dataset[0]
x.shape, y.shape

-> Loading word embeddings


(torch.Size([4, 100]), torch.Size([3]))

In [31]:
output = model(x.float())
output.shape

---
torch.Size([4, 100])
torch.Size([100])
torch.Size([3])
---


torch.Size([3])

In [32]:
loss = torch.nn.CrossEntropyLoss()

In [35]:
y.shape

torch.Size([3])

In [40]:
loss(output, y.argmax())

tensor(1.0386, grad_fn=<NllLossBackward0>)